In [164]:
import numpy as np
import cv2

In [165]:
#定义显示图片的函数cv_show
def cv_show(name, image):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(name,image.shape[0],image.shape[1])
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [166]:
#定义SIFT检测算法函数
def detectAndDescribe(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    descriptor=cv2.SIFT_create()
    (kps, features) = descriptor.detectAndCompute(gray, None)       #运用SIFT检测关键点
    kps = np.float32([kp.pt for kp in kps])
    return (kps, features)


In [167]:
#定义特征点匹配函数
def matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio = 0.75, reprojThresh = 4.0):
    matcher = cv2.BFMatcher()
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)      #运用KNN算法匹配
    matches = []
    for m in rawMatches:
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            matches.append((m[0].trainIdx, m[0].queryIdx))
    if len(matches) > 10:       #匹配关键点的对数必须大于10，才能进行拼接
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)      #记录透视矩阵，方便下一步进行透视变换
        return (matches, H, status)
    return None


In [168]:
#定义显示匹配点连线函数
def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        if s == 1:
            ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
            ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)     #对匹配的关键点进行连线
    cv_show("drawImg", vis)
    return vis


In [169]:
#定义拼接函数
def stitch(imageA,imageB, ratio=0.75, reprojThresh=4.0,showMatches=False):
    (kpsA, featuresA) = detectAndDescribe(imageA)
    (kpsB, featuresB) = detectAndDescribe(imageB)
    M = matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
    if M is None:
        return None     
    (matches, H, status) = M
    result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))       #将右边的图进行透视变化
    result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB       #将左边的图覆盖到已透视变换的上一步result中，得到未后处理的图像
    if showMatches:
        vis = drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
        return (result, vis)
    return result


In [170]:
#定义裁剪函数
def change_size(image):
    img = cv2.medianBlur(image,5)       #进行中值滤波
    b=cv2.threshold(img,15,255,cv2.THRESH_BINARY)          
    binary_image=b[1]              
    binary_image=cv2.cvtColor(binary_image,cv2.COLOR_BGR2GRAY)      
 
    x=binary_image.shape[0]
    y=binary_image.shape[1]
    edges_x=[]
    edges_y=[]
    for i in range(x):
        for j in range(y):
            if binary_image[i][j]==255:     #将像素值为255(即黑色)的点添加到边缘
             edges_x.append(i)
             edges_y.append(j)

    #寻找边缘点
    left=min(edges_x)               
    right=max(edges_x)              
    width=right-left                
    bottom=min(edges_y)             
    top=max(edges_y)                
    height=top-bottom               
 
    pre1_picture=image[left:left+width,bottom:bottom+height]        #进行裁剪       
    cv_show('result',pre1_picture)

In [171]:
# 读取图像
imageA = cv2.imread('data/6/2.png')
imageB = cv2.imread('data/6/1.png')
#得到关键点
(kpsA, featuresA) = detectAndDescribe(imageA)
(kpsB, featuresB) = detectAndDescribe(imageB)
#进行关键点的匹配并将其拼接
(matches,H,status) = matchKeypoints(kpsA, kpsB, featuresA, featuresB)
drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
result = stitch(imageA, imageB)

#后处理(去除黑边区域)
change_size(result)
